In [1]:
from pandas import DataFrame
import pandas as pd

try:
    from urllib import urlencode
except ImportError:
    from urllib.parse import urlencode

In [2]:
from jsonapi_client import Session, Filter

API_BASE = 'https://www.ebi.ac.uk/metagenomics/api/latest/'

In [3]:
study1 = 'SRP002480'

### List samples with biomes for the given study

Get study: https://www.ebi.ac.uk/metagenomics/api/latest/studies/SRP002480

List samples: https://www.ebi.ac.uk/metagenomics/api/latest/studies/SRP002480/samples


Fetch samples for the given study accession: https://www.ebi.ac.uk/metagenomics/api/latest/samples?study_accession=SRP002480


In [5]:
with Session(API_BASE) as s:
    study = s.get('studies', 'ERP009004').resource
    print('Study name:', study.study_name)
    print('Study abstract:', study.study_abstract)
    for biome in study.biomes:
        print('Biome:', biome.biome_name, biome.lineage)

Study name: Hydrocarbon Metagenomics Project
Study abstract: Metagenomics for Greener Production and Extraction of Hydrocarbon Energy:
Creating Opportunities for Enhanced Recovery with Reduced Environmental Impact
Biome: Freshwater root:Environmental:Aquatic:Freshwater
Biome: Marine root:Environmental:Aquatic:Marine
Biome: Soil root:Environmental:Terrestrial:Soil


In [6]:
def get_metadata(metadata, key):
    import html
    for m in metadata:
        if m['key'].lower() == key.lower():
            value = m['value']
            unit = html.unescape(m['unit']) if m['unit'] else ""
            return "{value} {unit}".format(value=value, unit=unit)
    return None

depth_label = 'geographic location (depth)'
temp_label = 'temperature'
df = DataFrame(columns=('sample name', 'biome', 'temperature', 'depth', 'longitude', 'latitude'))
df.index.name = 'accession'

with Session(API_BASE) as s:
    params = {
        'study_accession': 'ERP001736',
        'include': 'biome',
        'page_size': 100,
    }
    f = Filter(urlencode(params))
    for sample in s.iterate('samples', f):
        df.loc[sample.accession] = [
            sample.sample_name, sample.biome.id,
            get_metadata(sample.sample_metadata, temp_label),
            get_metadata(sample.sample_metadata, depth_label),
            sample.longitude, sample.latitude
        ]
df

JSONDecodeError: Unterminated string starting at: line 1 column 84574 (char 84573)